<a href="https://colab.research.google.com/github/busiri/busil/blob/main/3%EB%8B%A8%EA%B3%84(%EB%B6%84%EB%A6%AC_%EB%B0%8F_%EA%B2%B0%EC%B8%A1%EC%B9%98_%EC%B2%98%EB%A6%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 데이터 로드

In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm

df1 = pd.read_csv('/content/eda도입_winsorized.csv')
df2 = pd.read_csv('/content/eda성장_winsorized.csv')
df3 = pd.read_csv('/content/eda성숙_winsorized.csv')
df4 = pd.read_csv('/content/eda쇠퇴_winsorized.csv')

### 데이터 분리

In [55]:
def split_data(df:pd.DataFrame):
  X = df.drop('부실여부',axis=1)
  y = df['부실여부']
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
  train_df = pd.concat([X_train,y_train],axis=1)
  test_df = pd.concat([X_test,y_test],axis=1)
  return train_df, test_df

df_list = [df1,df2,df3,df4]
name_list = ['도입','성장','성숙','쇠퇴']
for df, name in zip(df_list,name_list):
 globals()[f"train_{name}"],globals()[f"test_{name}"] = split_data(df)

In [56]:
print(f"도입기 훈련 데이터 : {train_도입.shape}, 도입기 평가 데이터 : {test_도입.shape}")
print(f"성장기 훈련 데이터 : {train_성장.shape}, 성장기 평가 데이터 : {test_성장.shape}")
print(f"성숙기 훈련 데이터 : {train_성숙.shape}, 성숙기 평가 데이터 : {test_성숙.shape}")
print(f"쇠퇴기 훈련 데이터 : {train_쇠퇴.shape}, 쇠퇴기 평가 데이터 : {test_쇠퇴.shape}")

도입기 훈련 데이터 : (33851, 162), 도입기 평가 데이터 : (8463, 162)
성장기 훈련 데이터 : (43180, 162), 성장기 평가 데이터 : (10795, 162)
성숙기 훈련 데이터 : (54576, 162), 성숙기 평가 데이터 : (13644, 162)
쇠퇴기 훈련 데이터 : (19181, 162), 쇠퇴기 평가 데이터 : (4796, 162)


### 1년만 존재하는 데이터는 없앤다

In [57]:
group_counts = train_도입.groupby(['회사명', '거래소코드'])['회계년도'].count()
valid_keys = group_counts[group_counts >= 2].index

# 각 행의 (회사명, 거래소코드) 튜플이 valid_keys에 있는지 확인
train_도입 = train_도입[train_도입.set_index(['회사명', '거래소코드']).index.isin(valid_keys)]

In [58]:
group_counts = train_성장.groupby(['회사명', '거래소코드'])['회계년도'].count()
valid_keys = group_counts[group_counts >= 2].index

# 각 행의 (회사명, 거래소코드) 튜플이 valid_keys에 있는지 확인
train_성장 = train_성장[train_성장.set_index(['회사명', '거래소코드']).index.isin(valid_keys)]

In [59]:
group_counts = train_성숙.groupby(['회사명', '거래소코드'])['회계년도'].count()
valid_keys = group_counts[group_counts >= 2].index

# 각 행의 (회사명, 거래소코드) 튜플이 valid_keys에 있는지 확인
train_성숙 = train_성숙[train_성숙.set_index(['회사명', '거래소코드']).index.isin(valid_keys)]

In [60]:
group_counts = train_쇠퇴.groupby(['회사명', '거래소코드'])['회계년도'].count()
valid_keys = group_counts[group_counts >= 2].index

# 각 행의 (회사명, 거래소코드) 튜플이 valid_keys에 있는지 확인
train_쇠퇴 = train_쇠퇴[train_쇠퇴.set_index(['회사명', '거래소코드']).index.isin(valid_keys)]

In [61]:
print(train_도입.shape)
print(train_성장.shape)
print(train_성숙.shape)
print(train_쇠퇴.shape)

(21275, 162)
(32453, 162)
(44467, 162)
(8928, 162)


### 0이 30퍼센트 이상인 컬럼도 없앤다(train 기준)

In [62]:
# 범주형 컬럼 처리
object_cols = ['기업규모명', '통계청 한국표준산업분류 11차(대분류)', '상장폐지일', '대표이사변경여부', '수도권', '업력',
               '부실여부', 'IFRS', 'IFRS_CONN', 'GAAP', '회계년도연속여부']
train_list = [train_도입,train_성장,train_성숙,train_쇠퇴]
test_list = [test_도입,test_성장,test_성숙,test_쇠퇴]
for train, test in zip(train_list,test_list):
  for col in object_cols:
    train[col] = train[col].astype('object')
    test[col] = test[col].astype('object')

In [63]:
def check_zero(df):
  num_cols = df.select_dtypes(exclude='object').columns
  lis = []
  for col in num_cols:
    ratio = len(df[col].loc[df[col]==0])/len(df[col])
    if ratio > 0.3:
      lis.append(col)
  return lis

In [64]:
for train,test in zip(train_list,test_list):
  drop_cols = check_zero(train)
  train.drop(columns=drop_cols,inplace=True)
  test.drop(columns=drop_cols,inplace=True)

In [65]:
print(f"도입기 훈련 데이터 : {train_도입.shape}, 도입기 평가 데이터 : {test_도입.shape}")
print(f"성장기 훈련 데이터 : {train_성장.shape}, 성장기 평가 데이터 : {test_성장.shape}")
print(f"성숙기 훈련 데이터 : {train_성숙.shape}, 성숙기 평가 데이터 : {test_성숙.shape}")
print(f"쇠퇴기 훈련 데이터 : {train_쇠퇴.shape}, 쇠퇴기 평가 데이터 : {test_쇠퇴.shape}")

도입기 훈련 데이터 : (21275, 126), 도입기 평가 데이터 : (8463, 126)
성장기 훈련 데이터 : (32453, 136), 성장기 평가 데이터 : (10795, 136)
성숙기 훈련 데이터 : (44467, 135), 성숙기 평가 데이터 : (13644, 135)
쇠퇴기 훈련 데이터 : (8928, 126), 쇠퇴기 평가 데이터 : (4796, 126)


### 결측치 처리 : 회사 그룹별로 데이터가 있으면 회사 중앙값

## 수치형

In [66]:
numeric_cols1 = train_도입.select_dtypes(include=[np.number]).columns
numeric_cols2 = train_성장.select_dtypes(include=[np.number]).columns
numeric_cols3 = train_성숙.select_dtypes(include=[np.number]).columns
numeric_cols4 = train_쇠퇴.select_dtypes(include=[np.number]).columns
print(f"도입기 훈련 데이터 결측치 수 : {train_도입[numeric_cols1].isna().sum().sum()}, 도입기 평가 데이터 결측치 수 : {test_도입[numeric_cols1].isna().sum().sum()}")
print(f"성장기 훈련 데이터 결측치 수 : {train_성장[numeric_cols2].isna().sum().sum()}, 성장기 평가 데이터 결측치 수 : {test_성장[numeric_cols2].isna().sum().sum()}")
print(f"성숙기 훈련 데이터 결측치 수 : {train_성숙[numeric_cols3].isna().sum().sum()}, 성숙기 평가 데이터 결측치 수 : {test_성숙[numeric_cols3].isna().sum().sum()}")
print(f"쇠퇴기 훈련 데이터 결측치 수 : {train_쇠퇴[numeric_cols4].isna().sum().sum()}, 쇠퇴기 평가 데이터 결측치 수 : {test_쇠퇴[numeric_cols4].isna().sum().sum()}")

도입기 훈련 데이터 결측치 수 : 2477, 도입기 평가 데이터 결측치 수 : 1155
성장기 훈련 데이터 결측치 수 : 4816, 성장기 평가 데이터 결측치 수 : 1440
성숙기 훈련 데이터 결측치 수 : 5087, 성숙기 평가 데이터 결측치 수 : 1801
쇠퇴기 훈련 데이터 결측치 수 : 1317, 쇠퇴기 평가 데이터 결측치 수 : 653


In [67]:
# tqdm 설정
tqdm.pandas()

def fill_na(df):
    group_cols = ['회사명', '거래소코드']
    numeric_cols = df.select_dtypes(include='number').columns

    # 그룹별 중앙값 계산
    group_medians = df.groupby(group_cols)[numeric_cols].median()

    # 각 행에 대해 결측치를 중앙값으로 채움 (단, 중앙값이 NaN이면 그대로 둠)
    def fill_with_group_median(row):
        key = (row['회사명'], row['거래소코드'])
        if key in group_medians.index:
            for col in numeric_cols:
                if pd.isna(row[col]):
                    median_val = group_medians.loc[key, col]
                    if not pd.isna(median_val):
                        row[col] = median_val
        return row

    # 진행률 표시하면서 적용
    df = df.progress_apply(fill_with_group_median, axis=1)
    return df

# 학습/테스트 리스트
result = []
for train, test in zip(train_list, test_list):
    train = fill_na(train)
    test = fill_na(test)
    result.append(train)
    result.append(test)

100%|██████████| 4796/4796 [00:03<00:00, 1467.19it/s]


In [68]:
train_도입=result[0].fillna(0)
test_도입 =result[1].fillna(0)
train_성장=result[2].fillna(0)
test_성장=result[3].fillna(0)
train_성숙=result[4].fillna(0)
test_성숙=result[5].fillna(0)
train_쇠퇴=result[6].fillna(0)
test_쇠퇴=result[7].fillna(0)
print(f"도입기 훈련 데이터 결측치 수 : {train_도입[numeric_cols1].isna().sum().sum()}, 도입기 평가 데이터 결측치 수 : {test_도입[numeric_cols1].isna().sum().sum()}")
print(f"성장기 훈련 데이터 결측치 수 : {train_성장[numeric_cols2].isna().sum().sum()}, 성장기 평가 데이터 결측치 수 : {test_성장[numeric_cols2].isna().sum().sum()}")
print(f"성숙기 훈련 데이터 결측치 수 : {train_성숙[numeric_cols3].isna().sum().sum()}, 성숙기 평가 데이터 결측치 수 : {test_성숙[numeric_cols3].isna().sum().sum()}")
print(f"쇠퇴기 훈련 데이터 결측치 수 : {train_쇠퇴[numeric_cols4].isna().sum().sum()}, 쇠퇴기 평가 데이터 결측치 수 : {test_쇠퇴[numeric_cols4].isna().sum().sum()}")

도입기 훈련 데이터 결측치 수 : 0, 도입기 평가 데이터 결측치 수 : 0
성장기 훈련 데이터 결측치 수 : 0, 성장기 평가 데이터 결측치 수 : 0
성숙기 훈련 데이터 결측치 수 : 0, 성숙기 평가 데이터 결측치 수 : 0
쇠퇴기 훈련 데이터 결측치 수 : 0, 쇠퇴기 평가 데이터 결측치 수 : 0


## 범주형

In [69]:
# 기업규모명 삭제
train_도입.drop(['기업규모명','상장폐지일'],axis=1,inplace=True)
train_성장.drop(['기업규모명','상장폐지일'],axis=1,inplace=True)
train_성숙.drop(['기업규모명','상장폐지일'],axis=1,inplace=True)
train_쇠퇴.drop(['기업규모명','상장폐지일'],axis=1,inplace=True)
test_도입.drop(['기업규모명','상장폐지일'],axis=1,inplace=True)
test_성장.drop(['기업규모명','상장폐지일'],axis=1,inplace=True)
test_성숙.drop(['기업규모명','상장폐지일'],axis=1,inplace=True)
test_쇠퇴.drop(['기업규모명','상장폐지일'],axis=1,inplace=True)

In [70]:
object_cols1 = train_도입.select_dtypes('object').columns
object_cols2 = train_성장.select_dtypes('object').columns
object_cols3 = train_성숙.select_dtypes('object').columns
object_cols4 = train_쇠퇴.select_dtypes('object').columns
print(f"도입기 훈련 데이터 결측치 수 : {train_도입[object_cols1].isna().sum().sum()}, 도입기 평가 데이터 결측치 수 : {test_도입[object_cols1].isna().sum().sum()}")
print(f"성장기 훈련 데이터 결측치 수 : {train_성장[object_cols2].isna().sum().sum()}, 성장기 평가 데이터 결측치 수 : {test_성장[object_cols2].isna().sum().sum()}")
print(f"성숙기 훈련 데이터 결측치 수 : {train_성숙[object_cols3].isna().sum().sum()}, 성숙기 평가 데이터 결측치 수 : {test_성숙[object_cols3].isna().sum().sum()}")
print(f"쇠퇴기 훈련 데이터 결측치 수 : {train_쇠퇴[object_cols4].isna().sum().sum()}, 쇠퇴기 평가 데이터 결측치 수 : {test_쇠퇴[object_cols4].isna().sum().sum()}")

도입기 훈련 데이터 결측치 수 : 0, 도입기 평가 데이터 결측치 수 : 0
성장기 훈련 데이터 결측치 수 : 0, 성장기 평가 데이터 결측치 수 : 0
성숙기 훈련 데이터 결측치 수 : 0, 성숙기 평가 데이터 결측치 수 : 0
쇠퇴기 훈련 데이터 결측치 수 : 0, 쇠퇴기 평가 데이터 결측치 수 : 0


### 파생변수 추가하기

In [75]:
def add_derived_vars_no_lag(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 0) 로그 변환 지표 추가
    df['/ 로그매출액'] = np.log(df['매출액(천원)'].replace(0, np.nan))
    df['/ 로그자산']   = np.log(df['자산(천원)'].replace(0, np.nan))

    # 안전 나눗셈 도우미
    def safe_div(num, denom):
        return np.where(denom == 0, np.nan, num / denom)

    # 1) 안정성 비율
    df['/ 현금/총자산 비율'] = safe_div(df['현금및현금성자산(천원)'], df['자산(천원)'])
    df['/ 이익잉여금/총자산 비율'] = safe_div(df['이익잉여금(천원)'], df['자산(천원)'])
    df['/ 급여/총자산 비율'] = safe_div(df['급여(천원)'],df['자산(천원)'])

    # 2) 비용·레버리지·효율
    df['/ 급여/매출액 비율'] = safe_div(df['급여(천원)'],df['매출액(천원)'])
    df['/ 업력 효율지수'] = safe_div(df['매출액(천원)'],df['업력'])
    df['/ 매출원가/부채 비율'] = safe_div(df['매출원가(천원)'],df['부채(천원)'])
    df['/ 급여/세전순익 비율'] = safe_div(df['급여(천원)'],df['법인세비용차감전손익(천원)'])

    # 3) 추가 비용·레버리지 지표
    df['/ 금융비용부담률'] = safe_div(df['이자비용(천원)'], df['매출액(천원)'])

    # 4) 현금흐름 변수
    df['/ 영업CF/자산 비율'] = safe_div(df['영업현금흐름(천원)'],df['자산(천원)'])
    df['/ 투자CF/자산 비율'] = safe_div(df['투자활동으로 인한 현금흐름(*)(천원)'], df['자산(천원)'])
    df['/ 영업CF/유동부채 비율'] = safe_div(df['영업현금흐름(천원)'],df['유동부채(천원)'])
    total_borrow = df.get('총차입금(천원)', df['부채(천원)'])
    df['/ 재무CF/총차입금 비율'] = safe_div(df['재무활동으로 인한 현금흐름(*)(천원)'], total_borrow)

    # 5) 통합현금흐름 긍정여부
    cf_sum = (
        df['영업현금흐름(천원)']
      + df['투자활동으로 인한 현금흐름(*)(천원)']
      + df['재무활동으로 인한 현금흐름(*)(천원)']
    )
    df['/ 현금흐름통합_긍정여부'] = np.where(cf_sum > 0, 1, 0)

    # 6) 투하자본수익률 (영업이익 ÷ (자본+부채) ×100)
    total_cap = df['자본(천원)'] + df['부채(천원)']
    df['/ 투하자본수익률'] = safe_div(df['영업이익(손실)(천원)'], total_cap) * 100

    # 7) EBIT
    df['EBIT'] = df['법인세비용차감전손익(천원)'] + df['이자비용(천원)']
    df['EBIT / 자산'] = df['EBIT']/df['자산(천원)']
    df['EBIT / 매출액'] = df['EBIT']/df['매출액(천원)']
    df['EBIT / 영업현금흐름(천원)'] = df['EBIT']/df['영업현금흐름(천원)']

    # 8) 업력
    df['업력_범주12'] = df['업력'].map(lambda x : 1 if x>12 else 0)

    # 9) 무한대 → NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df

train_도입_파생 = add_derived_vars_no_lag(train_도입)
train_성장_파생 = add_derived_vars_no_lag(train_성장)
train_성숙_파생 = add_derived_vars_no_lag(train_성숙)
train_쇠퇴_파생 = add_derived_vars_no_lag(train_쇠퇴)
test_도입_파생 = add_derived_vars_no_lag(test_도입)
test_성장_파생 = add_derived_vars_no_lag(test_성장)
test_성숙_파생 = add_derived_vars_no_lag(test_성숙)
test_쇠퇴_파생 = add_derived_vars_no_lag(test_쇠퇴)

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(uf

In [76]:
# 파생변수에 사용된 컬럼 제거
derivate_col = ['매출액(천원)','자산(천원)','현금및현금성자산(천원)','이익잉여금(천원)','급여(천원)','업력',
'매출원가(천원)','부채(천원)','법인세비용차감전손익(천원)','이자비용(천원)','영업현금흐름(천원)',
'투자활동으로 인한 현금흐름(*)(천원)','유동부채(천원)','재무활동으로 인한 현금흐름(*)(천원)',
'영업이익(손실)(천원)','EBIT']

train_도입_파생.drop(columns=derivate_col,inplace=True)
train_성장_파생.drop(columns=derivate_col,inplace=True)
train_성숙_파생.drop(columns=derivate_col,inplace=True)
train_쇠퇴_파생.drop(columns=derivate_col,inplace=True)
test_도입_파생.drop(columns=derivate_col,inplace=True)
test_성장_파생.drop(columns=derivate_col,inplace=True)
test_성숙_파생.drop(columns=derivate_col,inplace=True)
test_쇠퇴_파생.drop(columns=derivate_col,inplace=True)

In [78]:
# 파일 보내기
train_도입_파생.to_csv('도입기_train.csv',index=False)
train_성장_파생.to_csv('성장기_train.csv',index=False)
train_성숙_파생.to_csv('성숙기_train.csv',index=False)
train_쇠퇴_파생.to_csv('쇠퇴기_train.csv',index=False)
test_도입_파생.to_csv('도입기_test.csv',index=False)
test_성장_파생.to_csv('성장기_test.csv',index=False)
test_성숙_파생.to_csv('성숙기_test.csv',index=False)
test_쇠퇴_파생.to_csv('쇠퇴기_test.csv',index=False)